In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import linregress
import seaborn as sns

In [ ]:
sns.set_style("ticks")

# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['figure.dpi'] = 120

# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 10
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 14
mpl.rc('text', usetex=False)

In [ ]:
output_dir = Path(snakemake.output.plots)

In [ ]:
output_dir

In [ ]:
output_dir.mkdir(exist_ok=True)

In [ ]:
df = pd.read_csv(snakemake.input.distances, sep="\t").dropna()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df.query("frequency > 0").copy()

In [ ]:
df.shape

In [ ]:
subclades = df["subclade"].drop_duplicates().sort_values().values

In [ ]:
n_subclades = len(subclades)

In [ ]:
n_subclades

In [ ]:
i = 1
with open(snakemake.input.color_schemes, "r", encoding="utf-8") as fh:
    for line in fh:
        if i == n_subclades:
            colors = line.strip().split("\t")
            break
            
        i = i + 1

In [ ]:
color_by_clade = dict(zip(subclades, colors))

In [ ]:
historical_clades = df["historicalclade"].drop_duplicates().sort_values().values

In [ ]:
n_historical_clades = len(historical_clades)

In [ ]:
i = 1
with open(snakemake.input.color_schemes, "r", encoding="utf-8") as fh:
    for line in fh:
        if i == n_historical_clades:
            historical_colors = line.strip().split("\t")
            break
            
        i = i + 1

In [ ]:
color_by_historical_clade = dict(zip(historical_clades, historical_colors))

In [ ]:
escape_score_attributes_to_plot = snakemake.params.escape_score_attributes_to_plot

In [ ]:
escape_score_attributes_to_plot

In [ ]:
attribute = "welsh_escape_nonnegative_scores_2-5_years_per_ha1"

In [ ]:
str(output_dir / f"distance_to_the_future_by_{attribute}_colored_by_subclade.pdf")

In [ ]:
correlations_per_attribute_and_season = []

for attribute in escape_score_attributes_to_plot:
    print(attribute)
    output_file = str(output_dir / Path(f"distance_to_the_future_by_{attribute}_colored_by_subclade.pdf"))
    
    g = sns.lmplot(
        df,
        x=attribute,
        y="weighted_distance_to_observed_future",
        hue="subclade",
        hue_order=subclades,
        palette=color_by_clade,
        col="season",
        col_wrap=3,
        fit_reg=False,
        height=6,
        scatter_kws={"alpha": 0.5},
    )

    g.set_axis_labels(
        f"{attribute}",
        "Weighted distance to observed future (AAs)",
    )

    x_values = np.arange(0, df[attribute].max(), 0.001)

    for season, season_ax in g.axes_dict.items():
        season_df = df[df["season"] == season]
        slope, intercept, r, p, se = linregress(
            season_df[attribute].values,
            season_df["weighted_distance_to_observed_future"].values,
        )
        intercept_sign = "+" if intercept >= 0 else "-"
        abs_intercept = np.abs(intercept)

        regression = f"y = {slope:.2f}x {intercept_sign} {abs_intercept:.2f}\nPearson's $R^2$={r**2:.2f}"

        season_ax.text(
            0.25,
            0.15,
            regression,
            horizontalalignment='center',
            verticalalignment='center',
            transform=season_ax.transAxes,
        )

        y_values = (slope * x_values) + intercept
        season_ax.plot(
            x_values,
            y_values,
            "-",
            color="#999999",
            zorder=-10,
        )

        season_ax.set_ylim(bottom=0.0)
        
        correlations_per_attribute_and_season.append({
            "attribute": attribute.replace("welsh_escape_", "").replace("_per_ha1", ""),
            "season": season,
            "pearson_r": r,
            "pearson_r2": r**2,
        })

    g.tight_layout()
    plt.show()
    plt.savefig(output_file, dpi=300)

In [ ]:
correlations_per_attribute_and_season_df = pd.DataFrame(correlations_per_attribute_and_season)

In [ ]:
correlations_per_attribute_and_season_df.sort_values("pearson_r2", ascending=False)

In [ ]:
attribute_order = correlations_per_attribute_and_season_df.groupby(
    "attribute"
).agg({
    "pearson_r2": "median"
}).sort_values(
    "pearson_r2",
    ascending=False,
).index.values

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)
ax = sns.stripplot(
    data=correlations_per_attribute_and_season_df,
    x="pearson_r2",
    y="attribute",
    hue="season",
    order=attribute_order,
    ax=ax,
    alpha=0.5,
)

ax = sns.boxplot(
    data=correlations_per_attribute_and_season_df,
    x="pearson_r2",
    y="attribute",
    order=attribute_order,
    fill=False,
    color="#999999",
    fliersize=0,
    ax=ax,
)

ax.legend(
    title="season",
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    borderaxespad=0.,
    frameon=False,
)

ax.set_xlabel("Pearson's $R^{2}$ between scaled escape score and distance to the future")
ax.set_ylabel("Dataset (scores and sera used)")

In [ ]:
attribute_order = correlations_per_attribute_and_season_df.groupby(
    "attribute"
).agg({
    "pearson_r": "median"
}).sort_values(
    "pearson_r",
    ascending=True,
).index.values

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)
ax = sns.stripplot(
    data=correlations_per_attribute_and_season_df,
    x="pearson_r",
    y="attribute",
    hue="season",
    order=attribute_order,
    ax=ax,
    alpha=0.5,
)

ax = sns.boxplot(
    data=correlations_per_attribute_and_season_df,
    x="pearson_r",
    y="attribute",
    order=attribute_order,
    fill=False,
    color="#999999",
    fliersize=0,
    ax=ax,
)

ax.legend(
    title="season",
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    borderaxespad=0.,
    frameon=False,
)

ax.set_xlabel("Pearson's $R$ between scaled escape score and distance to the future")
ax.set_ylabel("Dataset (scores and sera used)")

In [ ]:
seasons = df["season"].drop_duplicates().sort_values().values

In [ ]:
regression_placement_by_season = {
    "2020-10-01": (0.35, 0.15),
    "2021-02-01": (0.75, 0.6),
    "2021-10-01": (0.75, 0.75),
    "2022-02-01": (0.35, 0.5),
    "2022-10-01": (0.35, 0.15),
    "2023-02-01": (0.75, 0.75),
}

In [ ]:
for attribute in escape_score_attributes_to_plot:
    print(attribute)
    output_file = str(output_dir / Path(f"distance_to_the_future_by_{attribute}_colored_by_historical_clade.pdf"))
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 8), dpi=300)

    x_values = np.arange(0, df[attribute].max(), 0.001)

    for season, ax in zip(seasons, axes.flatten()):
        df_ax = df.query(f"season == '{season}'")

        slope, intercept, r, p, se = linregress(
            df_ax[attribute].values,
            df_ax["weighted_distance_to_observed_future"].values,
        )
        intercept_sign = "+" if intercept >= 0 else "-"
        abs_intercept = np.abs(intercept)

        regression = f"y = {slope:.2f}x {intercept_sign} {abs_intercept:.2f}\nPearson's $R^2$={r**2:.2f}"

        ax = sns.scatterplot(
            data=df_ax,
            x=attribute,
            y="weighted_distance_to_observed_future",
            hue="historicalclade",
            ax=ax,
            legend="brief",
            alpha=0.5,
        )
        ax.set_xlabel(f"{attribute}")
        ax.set_ylabel("Weighted distance to\nobserved future (AAs)")

        ax.text(
            regression_placement_by_season[season][0],
            regression_placement_by_season[season][1],
            regression,
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax.transAxes,
            fontsize=12,
        )

        y_values = (slope * x_values) + intercept
        ax.plot(
            x_values,
            y_values,
            "-",
            color="#999999",
            zorder=-10,
        )

        ax.set_ylim(bottom=0.0)

        ax.legend(
            frameon=False,
            title="clade",
        )

        ax.set_title(season)

    plt.tight_layout()
    plt.show()
    plt.savefig(output_file, dpi=300)

In [ ]:
lbi_regression_placement_by_season = {
    "2020-10-01": (0.35, 0.1),
    "2021-02-01": (0.75, 0.5),
    "2021-10-01": (0.75, 0.5),
    "2022-02-01": (0.35, 0.1),
    "2022-10-01": (0.75, 0.1),
    "2023-02-01": (0.75, 0.1),
}

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8), dpi=300)

x_values = np.arange(0, df["lbi"].max(), 0.001)

for season, ax in zip(seasons, axes.flatten()):
    df_ax = df.query(f"season == '{season}'")
    
    slope, intercept, r, p, se = linregress(
        df_ax["lbi"].values,
        df_ax["weighted_distance_to_observed_future"].values,
    )
    intercept_sign = "+" if intercept >= 0 else "-"
    abs_intercept = np.abs(intercept)
    
    regression = f"y = {slope:.2f}x {intercept_sign} {abs_intercept:.2f}\nPearson's $R^2$={r**2:.2f}"
    
    ax = sns.scatterplot(
        data=df_ax,
        x="lbi",
        y="weighted_distance_to_observed_future",
        hue="historicalclade",
        ax=ax,
        legend="brief",
        alpha=0.5,
    )
    ax.set_xlabel("Local branching index (LBI)")
    ax.set_ylabel("Weighted distance to\nobserved future (AAs)")

    ax.text(
        lbi_regression_placement_by_season[season][0],
        lbi_regression_placement_by_season[season][1],
        regression,
        horizontalalignment='center',
        verticalalignment='center',
        transform=ax.transAxes,
        fontsize=12,
    )
    
    y_values = (slope * x_values) + intercept
    ax.plot(
        x_values,
        y_values,
        "-",
        color="#999999",
        zorder=-10,
    )
    
    ax.set_ylim(bottom=0.0)
    
    ax.legend(
        frameon=False,
        title="clade",
    )
    
    ax.set_title(season)

plt.tight_layout()
plt.savefig(snakemake.output.distances_by_historical_clade_and_lbi, dpi=300)